In [9]:
import requests
import pandas as pd
import os
import re
from pandas.io.json import json_normalize
import json

爬取歌单名称：华语评论1W＋

歌单共计2000余首华语评论1000+的歌曲，涵盖了较老的歌和较新的歌，包罗万象

歌单作者id：133511246

歌单id：728958078

API:/playlist/track/all?id=728958078&limit=50&offset=0

首先先获取歌单里面所有歌曲的id

In [5]:
url = "http://127.0.0.1:3000/playlist/track/all?id=728958078&limit=10&offset=0"
response = requests.request("GET", url).json()

In [6]:
response

{'songs': [{'name': '危险派对',
   'id': 1877996649,
   'pst': 0,
   't': 0,
   'ar': [{'id': 12236125, 'name': '王以太', 'tns': [], 'alias': []},
    {'id': 1172023, 'name': '刘至佳', 'tns': [], 'alias': []}],
   'alia': [],
   'pop': 100,
   'st': 0,
   'rt': '',
   'fee': 8,
   'v': 27,
   'crbt': None,
   'cf': '',
   'al': {'id': 133202644,
    'name': '危险派对',
    'picUrl': 'https://p2.music.126.net/GpnLproqUUyc4xmYKpRFcQ==/109951166516282895.jpg',
    'tns': [],
    'pic_str': '109951166516282895',
    'pic': 109951166516282900},
   'dt': 180571,
   'h': {'br': 320000, 'fid': 0, 'size': 7225005, 'vd': -57704, 'sr': 48000},
   'm': {'br': 192000, 'fid': 0, 'size': 4335021, 'vd': -55129, 'sr': 48000},
   'l': {'br': 128000, 'fid': 0, 'size': 2890029, 'vd': -53450, 'sr': 48000},
   'sq': {'br': 977692, 'fid': 0, 'size': 22067908, 'vd': -57699, 'sr': 48000},
   'hr': {'br': 1747706,
    'fid': 0,
    'size': 39448232,
    'vd': -57699,
    'sr': 48000},
   'a': None,
   'cd': '01',
   'no': 1,

In [28]:
def GetSongs(offset):
    url_ = "http://127.0.0.1:3000/playlist/track/all?id=728958078&limit=1000&offset=" + str(offset)
    data_ = requests.request("GET", url_).json()
    df_ = pd.json_normalize(data_['songs'])
    return df_['id']

In [37]:
list1 = GetSongs(0).tolist()
list2 = GetSongs(1000).tolist()
list3 = GetSongs(2000).tolist()
list1.extend(list2)
list1.extend(list3)

In [56]:
len(list1)

2213

In [58]:
list_test = list1[1:10]

经过复核歌单确实有2213首歌曲。接下来按照表单`list1`来爬取这中间每一首歌的热度最高的五条评论。

API:/comment/new?type=0&id=1877996649&sortType=2&pageSize=5

In [53]:
# 获取歌曲前五条热评
def GetComments(id):
    url_ = "http://127.0.0.1:3000/comment/new?type=0&id=" + str(id) + "&sortType=2&pageSize=5"
    data_ = requests.request("GET", url_).json()
    COLUMNS = ["user.userId","commentId", "user.nickname", "content", "timeStr", "likedCount", 'user.isHug', 'replyCount', 'authStatus']
    df_ = pd.json_normalize(data_['data']["comments"])
    return df_[COLUMNS]

In [60]:
def get_all_comments(song_list):
    ans = GetComments(song_list[0])
    for i in range(1,len(song_list)):
        new_df = GetComments(song_list[i])
        ans = pd.concat([ans, new_df])
    return ans

In [63]:
hot_comments = get_all_comments(list1)

In [65]:
hot_comments.to_csv("hot_comments.csv",encoding='utf_8_sig')

In [64]:
hot_comments

,user.userId,commentId,user.nickname,content,timeStr,likedCount,ipLocation.location,time,user.isHug,replyCount
0,3291743001,5428603245,玛卡巴卡在阿巴阿巴,不行啊！这歌也太让人快乐了！洗澡的时候我直接扭成龙卷风！！！,2021-10-01,95696,,1633086750426,False,1688
1,267627817,5427142964,皮崽-承蒙厚爱,这派对也太危险了，男孩子在外面玩一定要好好保护自己[惊恐],2021-09-29,68626,,1632903869145,False,468
2,428956762,5419339232,保久伽壹,“右手托住你的心跳” 怎么……托😬,2021-09-17,61605,,1631809126892,False,2304
3,1984646294,5446242912,阿泽潮音,你说你大姨妈走了 让我晚上去你家 宝 守灵这活哥真干不了啊,2021-10-27,44294,,1635309922447,False,629
4,287711061,5420069620,等X的新歌,托住你的心跳这句可太高情商了,2021-09-18,37328,,1631941803647,False,221
...,...,...,...,...,...,...,...,...,...,...
0,50277131,47624511,SUN一茗,相信这首歌会火！因为朱亚文，因为马伊琍。也因为我跟女朋友异地恋965天了，张倩，你看到了的话...,2015-11-18,177242,,1447826224390,False,2305
1,106889746,56906244,双鱼座Y丫,北上广不相信眼泪 江浙沪不相信邮费 辽吉可以不相信喝醉 京津冀你得有个肺,2015-12-27,95755,,1451199668082,False,435
2,104611195,60038537,没有永远只有,半夜女朋友要吃酸辣粉，煮好粉发现家里没有醋了，只放了辣椒。我端过去叫她凑合吃，她大发雷霆：“...,2016-01-08,76191,,1452247724076,False,447
3,59942873,47187897,小兔子乖乖门就是不开,高中语文老师曾说，这世界所有的单纯和善良都可以从眼睛里看出来，从她的眼中我看到是充斥着泪水的...,2015-11-16,65313,,1447657330523,False,111


In [66]:
# 获取歌曲前15条最新评论
def GetNewComments(id):
    url_ = "http://127.0.0.1:3000/comment/new?type=0&id=" + str(id) + "&sortType=3&pageSize=15"
    data_ = requests.request("GET", url_).json()
    COLUMNS = ["user.userId","commentId", "user.nickname", "content", "timeStr", "likedCount", 'user.isHug', 'replyCount']
    df_ = pd.json_normalize(data_['data']["comments"])
    return df_[COLUMNS]

In [67]:
def get_all_new_comments(song_list):
    ans = GetNewComments(song_list[0])
    for i in range(1,len(song_list)):
        new_df = GetNewComments(song_list[i])
        ans = pd.concat([ans, new_df])
    return ans

In [ ]:
new_comments = get_all_new_comments(list1)

In [ ]:
new_comments.to_csv("new_comments.csv",encoding='utf_8_sig')